In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# -*- coding: utf-8 -*-
'''
Use TextCNN to learn the feature of COURT_HELD text

# 改为关键词匹配
reason topics we have:
	标签、配料表、外包装违规（包括虚假、夸大信息，格式、名称规范问题等等）;
	假冒产品（假酒、假保健品之类的）;
	保质期、生产日期;
	原材料、添加剂;
	商标;
	出入境检验、检疫证明;
	生产许可证、生产标准、证明文件;
	餐饮食品安全卫生标准;（只限餐饮）
	是否为消费者（有些人是专业打假人）;
	进口食品相关产品尚无国家标准（对尚无国家标准的食品或相关产品未做安全性评估就销售）;
	产品质检不合格、质量有问题;
	不明确（可以当做干扰项去掉）
标签优先提取
'''
import os
import numpy as np
import pandas as pd
import torchtext
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
from torchtext.legacy import data
from sklearn.metrics import classification_report

In [4]:
path1 = "/content/drive/MyDrive/Colab Notebooks/textcnn/dataset/textcnn_dataset.csv"
path2 = "/content/drive/MyDrive/Colab Notebooks/textcnn/dataset/type1_2014_1-2-3-4-5-6-7-8-9-10-11-12_nps.csv"

all_classes = pd.read_csv(path1, encoding="utf-8")
splitted_text = pd.read_csv(path2, encoding="utf-8")
all_classes = all_classes[~all_classes['reason_topic'].isin(["不明确", "unknown"])][['id', 'reason_topic']]
splitted_text = splitted_text[['id', 'content', 'topic_phrases', 'phrase_vectors']]
all_cases = all_classes.merge(splitted_text, on="id")
all_cases['reason_topic'] = all_cases['reason_topic'].apply(lambda row: row.split(";")[0])
all_cases.rename(columns={"reason_topic": "label"}, inplace=True)

labels = list(all_cases['label'].unique())
print("All labels in this dataset are: ", labels)

All labels in this dataset are:  ['原材料、添加剂', '产品质检不合格、质量有问题', '标签、配料表、外包装违规', '进口食品相关产品尚无国家标准', '保质期、生产日期', '生产许可证、生产标准、证明文件', '假冒产品', '餐饮食品安全卫生标准', '出入境检验、检疫证明', '商标', '是否为消费者']


In [5]:
Counter(list(all_cases.label))
#存在有些标签数据量过少的现象

Counter({'产品质检不合格、质量有问题': 21,
         '保质期、生产日期': 64,
         '假冒产品': 13,
         '出入境检验、检疫证明': 5,
         '原材料、添加剂': 50,
         '商标': 1,
         '是否为消费者': 1,
         '标签、配料表、外包装违规': 139,
         '生产许可证、生产标准、证明文件': 41,
         '进口食品相关产品尚无国家标准': 3,
         '餐饮食品安全卫生标准': 52})

In [6]:
# 先去掉商标，是否为消费者，进口食品相关产品尚无国家标准这三个类别，因为数据量太小了
all_cases = all_cases[~all_cases['label'].isin(["商标", "是否为消费者", "进口食品相关产品尚无国家标准"])]
print(Counter(list(all_cases.label)))
all_cases.head()

Counter({'标签、配料表、外包装违规': 139, '保质期、生产日期': 64, '餐饮食品安全卫生标准': 52, '原材料、添加剂': 50, '生产许可证、生产标准、证明文件': 41, '产品质检不合格、质量有问题': 21, '假冒产品': 13, '出入境检验、检疫证明': 5})


,id,label,content,topic_phrases,phrase_vectors
0,57ab9058c2265c28a560195d,原材料、添加剂,上述认定 证据 当事人 陈述 本院 认定 事实 2012年8月16日 喻忠 淘宝网 天猫商城...,"['上述认定', '证据', '当事人', '本院', '事实', '淘宝网天猫商城', '...",[b'~\x8c\x1f@\x88\xf4\x17\xc0\x98\xf5B\xbf;S0?...
1,57baba28c2265c5f452d2cef,产品质检不合格、质量有问题,"本院认为,原告购买的是被告 散装茶叶 选用 被告 店 市场 流行 通用包装盒 装 茶 原告 ...","['散装茶叶', '市场', '通用包装盒', '收据', '当庭陈述', '质量安全管理'...",[b'~\xae\x92?\xe9\xd9(@\x9a*\xec?Wx\x1f\xbe\x1...
2,57abb74ac2265c258984430a,标签、配料表、外包装违规,"本院认为,食品安全标准 应 包括 食品安全 营养 标签 标识 说明书 食品经营者 查验 商品...","['食品安全', '营养', '标签', '标识', '食品经营者', '商品合格', '证...",[b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0...
4,57baba99c2265c5f452d3090,产品质检不合格、质量有问题,"本院认为,原告购买的是被告 散装茶叶 选用 被告 店 市场 流行 通用包装盒 装 茶 原告 ...","['散装茶叶', '市场', '通用包装盒', '收据', '当庭陈述', '质量安全管理'...",[b'~\xae\x92?\xe9\xd9(@\x9a*\xec?Wx\x1f\xbe\x1...
6,57baba8fc2265c5f452d303f,产品质检不合格、质量有问题,"本院认为,原告购买的是被告 散装茶叶 选用 被告 店 市场 流行 通用包装盒 装 茶 原告 ...","['散装茶叶', '市场', '通用包装盒', '收据', '当庭陈述', '质量安全管理'...",[b'~\xae\x92?\xe9\xd9(@\x9a*\xec?Wx\x1f\xbe\x1...


In [7]:
# 每个类别都按照311划分为train validation和test，将这三类文件输出
textcnn_bpath = "/content/drive/MyDrive/Colab Notebooks/textcnn/dataset"
train = pd.DataFrame(columns=["label", "content"])
val = pd.DataFrame(columns=["label", "content"])
test = pd.DataFrame(columns=["label", "content"])
for i in list(all_cases.label.unique()):
    print(i)
    train_len = int(len(all_cases[all_cases['label']==i]) * 0.6)
    val_len = int(len(all_cases[all_cases['label']==i]) * 0.2)
    # test_len = len(all_cases[all_cases['label']==i]) * 0.2
    print(train_len, val_len)
    train = train.append(all_cases[all_cases['label']==i][:train_len][['label', 'content']], ignore_index=True)
    val = val.append(all_cases[all_cases['label']==i][train_len:train_len+val_len][['label', 'content']], ignore_index=True)
    test = test.append(all_cases[all_cases['label']==i][train_len+val_len:][['label', 'content']], ignore_index=True)
with open(textcnn_bpath+"train.tsv",'w') as write_tsv:
    write_tsv.write(train.to_csv(sep='\t', index=False))
with open(textcnn_bpath+"val.tsv",'w') as write_tsv:
    write_tsv.write(val.to_csv(sep='\t', index=False))
with open(textcnn_bpath+"test.tsv",'w') as write_tsv:
    write_tsv.write(test.to_csv(sep='\t', index=False))

原材料、添加剂
30 10
产品质检不合格、质量有问题
12 4
标签、配料表、外包装违规
83 27
保质期、生产日期
38 12
生产许可证、生产标准、证明文件
24 8
假冒产品
7 2
餐饮食品安全卫生标准
31 10
出入境检验、检疫证明
3 1


In [8]:
#声明一个Field对象，对象里面填的就是需要对文本进行哪些操作，比如这里lower=True英文大写转小写,tokenize=cut对于文本分词采用之前定义好的cut函数，sequence=True表示输入的是一个sequence类型的数据，还有其他更多操作可以参考文档
TEXT = data.Field(sequential=True,lower=True)
#声明一个标签的LabelField对象，sequential=False表示标签不是sequence，dtype=torch.int64标签转化成整形
LABEL = data.LabelField(sequential=False, dtype=torch.int64)

#这里主要是告诉torchtext需要处理哪些数据，这些数据存放在哪里，TabularDataset是一个处理scv/tsv的常用类
train_dataset, dev_dataset, test_dataset = data.TabularDataset.splits(
      path=textcnn_bpath,  #文件存放路径
      format='tsv',   #文件格式
      skip_header=True,  #是否跳过表头，我这里数据集中没有表头，所以不跳过
      train='train.tsv',
      validation='val.tsv',
      test='test.tsv',
      fields=[('label',LABEL),('content',TEXT)] # 定义数据对应的表头
)

In [9]:
# 预训练词向量
pretrained_name = 'sgns.baidubaike.bigram-char' # 预训练词向量文件名
pretrained_path = textcnn_bpath #预训练词向量存放路径
vectors = torchtext.vocab.Vectors(name=pretrained_name, cache=pretrained_path)

In [10]:
# 建立词表
TEXT.build_vocab(train_dataset,
                 dev_dataset,
                 test_dataset,
                 vectors=vectors)
LABEL.build_vocab(train_dataset,
                  dev_dataset,
                  test_dataset)

In [13]:
# 查看词表信息
print("词表中的词汇总量为： ", len(TEXT.vocab))
print("词向量维度为： ", TEXT.vocab.vectors.shape)
print(TEXT.vocab.stoi['过期食品'])

词表中的词汇总量为：  8621
词向量维度为：  torch.Size([8621, 300])
544


In [13]:
#生成迭代器
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
        (train_dataset, dev_dataset, test_dataset), #需要生成迭代器的数据集
        batch_sizes=(128, 128, 128), # 每个迭代器分别以多少样本为一个batch
        sort_key=lambda x: len(x.content) #按什么顺序来排列batch，这里是以句子的长度，就是上面说的把句子长度相近的放在同一个batch里面
        )

In [16]:
# TextCNN建模
class TextCNN(nn.Module):
    def __init__(self,
                 class_num, # 最后输出的种类数
                 filter_sizes, # 卷积核的长也就是滑动窗口的长
                 filter_num,   # 卷积核的数量
                 vocabulary_size, # 词表的大小
                 embedding_dimension, # 词向量的维度
                 vectors, # 词向量
                 dropout): # dropout率
        super(TextCNN, self).__init__() # 继承nn.Module

        chanel_num = 1  # 通道数，也就是一篇文章一个样本只相当于一个feature map

        self.embedding = nn.Embedding(vocabulary_size, embedding_dimension) # 嵌入层
        self.embedding = self.embedding.from_pretrained(vectors) #嵌入层加载预训练词向量

        self.convs = nn.ModuleList(
            [nn.Conv2d(chanel_num, filter_num, (fsz, embedding_dimension)) for fsz in filter_sizes])  # 卷积层
        self.dropout = nn.Dropout(dropout) # dropout
        self.fc = nn.Linear(len(filter_sizes) * filter_num, class_num) #全连接层

    def forward(self, x):
        # x维度[句子长度,一个batch中所包含的样本数] 例:[3451,128]
        x = self.embedding(x) # #经过嵌入层之后x的维度，[句子长度,一个batch中所包含的样本数,词向量维度] 例：[3451,128,300]
        x = x.permute(1,0,2) # permute函数将样本数和句子长度换一下位置，[一个batch中所包含的样本数,句子长度,词向量维度] 例：[128,3451,300]
        x = x.unsqueeze(1) # # conv2d需要输入的是一个四维数据，所以新增一维feature map数 unsqueeze(1)表示在第一维处新增一维，[一个batch中所包含的样本数,一个样本中的feature map数，句子长度,词向量维度] 例：[128,1,3451,300]
        x = [conv(x) for conv in self.convs] # 与卷积核进行卷积，输出是[一个batch中所包含的样本数,卷积核数，句子长度-卷积核size+1,1]维数据,因为有[3,4,5]三张size类型的卷积核所以用列表表达式 例：[[128,16,3459,1],[128,16,3458,1],[128,16,3457,1]]
        x = [sub_x.squeeze(3) for sub_x in x]#squeeze(3)判断第三维是否是1，如果是则压缩，如不是则保持原样 例：[[128,16,3459],[128,16,3458],[128,16,3457]]
        x = [F.relu(sub_x) for sub_x in x] # ReLU激活函数激活，不改变x维度
        x = [F.max_pool1d(sub_x,sub_x.size(2)) for sub_x in x] # 池化层，根据之前说的原理，max_pool1d要取出每一个滑动窗口生成的矩阵的最大值，因此在第二维上取最大值 例：[[128,16,1],[128,16,1],[128,16,1]]
        x = [sub_x.squeeze(2) for sub_x in x] # 判断第二维是否为1，若是则压缩 例：[[128,16],[128,16],[128,16]]
        x = torch.cat(x, 1) # 进行拼接，例：[128,48]
        x = self.dropout(x) # 去除掉一些神经元防止过拟合，注意dropout之后x的维度依旧是[128,48]，并不是说我dropout的概率是0.5，去除了一半的神经元维度就变成了[128,24]，而是把x中的一些神经元的数据根据概率全部变成了0，维度依旧是[128,48]
        logits = self.fc(x) # 全接连层 例：输入x是[128,48] 输出logits是[128,10]
        return logits

In [17]:
# 模型训练
class_num = len(LABEL.vocab) # 类别数目
filter_size = [3,4,5]  # 卷积核种类数
filter_num=16   # 卷积核数量
vocab_size = len(TEXT.vocab) # 词表大小
embedding_dim = TEXT.vocab.vectors.size()[-1] # 词向量维度
vectors = TEXT.vocab.vectors # 词向量
dropout=0.5
learning_rate = 0.001  # 学习率
epochs = 300   # 迭代次数
save_dir = '/content/drive/MyDrive/Colab Notebooks/textcnn/model' # 模型保存路径
steps_show = 10   # 每10步查看一次训练集loss和mini batch里的准确率
steps_eval = 100  # 每100步测试一下验证集的准确率
early_stopping = 1000  # 若发现当前验证集的准确率在1000步训练之后不再提高 一直小于best_acc,则提前停止训练

textcnn_model = TextCNN(class_num=class_num,
        filter_sizes=filter_size,
        filter_num=filter_num,
        vocabulary_size=vocab_size,
        embedding_dimension=embedding_dim,
        vectors=vectors,
        dropout=dropout)

In [11]:
# 模型验证方法
def dev_eval(dev_iter, model):
    model.eval()
    corrects, avg_loss = 0, 0
    for batch in dev_iter:
        feature, target = batch.content, batch.label
        if torch.cuda.is_available():
            feature, target = feature.cuda(), target.cuda()
        logits = model(feature)
        # print(logits, target)
        loss = F.cross_entropy(logits, target)
        avg_loss += loss.item()
        pred_tags = torch.max(logits, 1)
        corrects += (torch.max(logits, 1)
                    [1].view(target.size()).data == target.data).sum()
    size = len(dev_iter.dataset)
    avg_loss /= size
    accuracy = 100.0 * corrects / size
    print('\nEvaluation - loss: {:.6f}  acc: {:.4f}%({}/{}) \n'.format(avg_loss,
                                                                          accuracy,
                                                                          corrects,
                                                                          size))
    return accuracy, target, pred_tags

In [19]:
# 定义模型保存函数
def save(model, save_dir, steps):
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    save_path = 'bestmodel_steps{}.pt'.format(steps)
    save_bestmodel_path = os.path.join(save_dir, save_path)
    torch.save(model.state_dict(), save_bestmodel_path)

In [20]:
def train(train_iter, dev_iter, model):

    if torch.cuda.is_available(): # 判断是否有GPU，如果有把模型放在GPU上训练，速度质的飞跃
        model.cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 梯度下降优化器，采用Adam
    steps = 0
    best_acc = 0
    last_step = 0
    model.train()
    for epoch in range(1, epochs + 1):
        for batch in train_iter:
            feature, target = batch.content, batch.label
            if torch.cuda.is_available(): # 如果有GPU将特征更新放在GPU上
                feature,target = feature.cuda(),target.cuda()
            optimizer.zero_grad() # 将梯度初始化为0，每个batch都是独立训练地，因为每训练一个batch都需要将梯度归零
            logits = model(feature)
            loss = F.cross_entropy(logits, target) # 计算损失函数 采用交叉熵损失函数
            loss.backward()  # 反向传播
            optimizer.step() # 放在loss.backward()后进行参数的更新
            steps += 1
            if steps % steps_show == 0: # 每训练多少步计算一次准确率，我这边是100，可以自己修改
                corrects = (torch.max(logits, 1)[1].view(target.size()).data == target.data).sum() # logits是[128,10],torch.max(logits, 1)也就是选出第一维中概率最大的值，输出为[128,1],torch.max(logits, 1)[1]相当于把每一个样本的预测输出取出来，然后通过view(target.size())平铺成和target一样的size (128,),然后把与target中相同的求和，统计预测正确的数量
                train_acc = 100.0 * corrects / batch.batch_size # 计算每个mini batch中的准确率
                print('steps:{} - loss: {:.6f}  acc:{:.4f}'.format(
                  steps,
                  loss.item(),
                  train_acc))

            if steps % steps_eval == 0: # 每训练100步进行一次验证
                dev_acc, target, pred_tags = dev_eval(dev_iter,model)
                print("dev_acc: ", dev_acc)
                if dev_acc > best_acc :
                    best_acc = dev_acc
                    last_step = steps
                    print('Saving best model, acc: {:.4f}%\n'.format(best_acc))
                    save(model, save_dir, steps)
                else:
                    if steps - last_step >= early_stopping:
                        print('\n提前停止于 {} steps, acc: {:.4f}%'.format(last_step, best_acc))
                        raise KeyboardInterrupt

In [21]:
#训练
train(train_iter, dev_iter, textcnn_model)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


steps:10 - loss: 1.440770  acc:49.0000
steps:20 - loss: 1.157834  acc:67.0000
steps:30 - loss: 0.972232  acc:69.5312
steps:40 - loss: 0.827206  acc:77.3438
steps:50 - loss: 0.695821  acc:81.0000
steps:60 - loss: 0.542120  acc:88.0000
steps:70 - loss: 0.585213  acc:82.8125
steps:80 - loss: 0.617400  acc:83.0000
steps:90 - loss: 0.512736  acc:89.0000
steps:100 - loss: 0.410736  acc:92.0000

Evaluation - loss: 0.012838  acc: 70.2703%(52/74) 

dev_acc:  tensor(70.2703, device='cuda:0')
Saving best model, acc: 70.2703%

steps:110 - loss: 0.220794  acc:97.0000
steps:120 - loss: 0.155750  acc:98.4375
steps:130 - loss: 0.124715  acc:98.4375
steps:140 - loss: 0.100286  acc:100.0000
steps:150 - loss: 0.065298  acc:100.0000
steps:160 - loss: 0.095035  acc:98.0000
steps:170 - loss: 0.067119  acc:99.0000
steps:180 - loss: 0.086792  acc:97.0000
steps:190 - loss: 0.040254  acc:99.2188
steps:200 - loss: 0.042255  acc:99.0000

Evaluation - loss: 0.012553  acc: 68.9189%(51/74) 

dev_acc:  tensor(68.9189

In [22]:
dev_acc, target, pred_tags = dev_eval(test_iter, textcnn_model)


Evaluation - loss: 0.013752  acc: 71.0843%(59/83) 



In [23]:
print(classification_report(pred_tags[1].cpu(), target.cpu()))

              precision    recall  f1-score   support

           0       0.93      0.66      0.77        41
           1       0.93      0.68      0.79        19
           2       1.00      1.00      1.00        11
           3       0.50      0.71      0.59         7
           4       0.33      0.75      0.46         4
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0

    accuracy                           0.71        83
   macro avg       0.46      0.48      0.45        83
weighted avg       0.86      0.71      0.77        83



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print(pred_tags[1].cpu())
for i in pred_tags[1].cpu()[:]:
  print(LABEL.vocab.itos[i])

tensor([0, 1, 3, 0, 3, 1, 0, 1, 3, 0, 0, 1, 0, 1, 0, 3, 1, 0, 5, 1, 0, 0, 0, 1,
        4, 1, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 1, 0, 0, 0, 3, 1, 0, 3, 1,
        0, 1, 0, 1, 1, 3, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1])
标签、配料表、外包装违规
保质期、生产日期
原材料、添加剂
标签、配料表、外包装违规
原材料、添加剂
保质期、生产日期
标签、配料表、外包装违规
保质期、生产日期
原材料、添加剂
标签、配料表、外包装违规
标签、配料表、外包装违规
保质期、生产日期
标签、配料表、外包装违规
保质期、生产日期
标签、配料表、外包装违规
原材料、添加剂
保质期、生产日期
标签、配料表、外包装违规
产品质检不合格、质量有问题
保质期、生产日期
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表、外包装违规
保质期、生产日期
生产许可证、生产标准、证明文件
保质期、生产日期
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表、外包装违规
生产许可证、生产标准、证明文件
生产许可证、生产标准、证明文件
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表、外包装违规
生产许可证、生产标准、证明文件
标签、配料表、外包装违规
标签、配料表、外包装违规
保质期、生产日期
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表、外包装违规
原材料、添加剂
保质期、生产日期
标签、配料表、外包装违规
原材料、添加剂
保质期、生产日期
标签、配料表、外包装违规
保质期、生产日期
标签、配料表、外包装违规
保质期、生产日期
保质期、生产日期
原材料、添加剂
标签、配料表、外包装违规
标签、配料表、外包装违规
保质期、生产日期
标签、配料表、外包装违规
保质期、生产日期
保质期、生产日期
标签、配料表、外包装违规
标签、配料表、外包装违规
标签、配料表

In [75]:
# TODO
# # 使用模型对现有数据做预测
# # 以2015年的第一类数据为例预测一下
# def pred(dev_iter, model):
#   for batch in dev_iter:
#         feature, target = batch.content, batch.label
#         if torch.cuda.is_available():
#             feature, target = feature.cuda(), target.cuda()
#         logits = model(feature)
#         # print(logits, target)
#         loss = F.cross_entropy(logits, target)
#         avg_loss += loss.item()
#         pred_tags = torch.max(logits, 1)
#         corrects += (torch.max(logits, 1)
#                     [1].view(target.size()).data == target.data).sum()